Hello Everybody, this Notebook will be used for my final project in Applied Data Science

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [5]:
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


# Business Understanding

The initial phase is to understand the project's objective from the business or application perspective. Then, you need to translate this knowledge into a machine learning problem with a preliminary plan to achieve the objectives.

Say you are driving to another city for work or to visit some friends. It is rainy and windy, and on the way, you come across a terrible traffic jam on the other side of the highway. Long lines of cars barely moving. As you keep driving, police car start appearing from afar shutting down the highway. Oh, it is an accident and there's a helicopter transporting the ones involved in the crash to the nearest hospital. They must be in critical condition for all of this to be happening.

Imagine we were able to find a way of predicting the severity of a car accident. In other words i want to predict the possibiity of you getting into a car accident and predict the possible severity of the accident on top. In order to make such predicitons i would take different attributes like weather or road conditions into account. In the end, given the prediction, you could drive more carefully or if possible even change you travel route.

Our Target audience is the SDOT Traffic Management Division. They could use the data they are already collecting and analyze which are the main drivers for accidents happening in certain areas. On top it is possible to calculate the influence of the main factors that lead to accidents. Knowing the main factors and their influence the objective is to define measurements that improve for example road conditions or light conditions. Lets say our analysis shows that severe accidents happen because of bad light conditions in a certain area the SDOT can take actions to improve these conditions and hopefully lower the future number of accidents.

So in clonclusion the furhter analysis should help SDOT to identify the main drivers and their influence. And of course the SDOT's main objective should be to lower the number and the severity of accidents. 

# Analytical Approach

To accurately build a model to prevent future accidents and/or reduce their severity, we will use the following attributes — ADDRTYPE, WEATHER, ROADCOND, VEHCOUNT, PERSONCOUNT

In the first step we will graphically analyse (e.g. using bar charts) the data and visualize how the attributes relate to the number and severity of accidents. Havi

# Data Understanding

In this phase, you need to collect or extract the dataset from various sources such as csv file or SQL database. Then, you need to determine the attributes (columns) that you will use to train your machine learning model. Also, you will assess the condition of chosen attributes by looking for trends, certain patterns, skewed information, correlations, and so on.

To adress the issue described above i will use a DataSet provided by the SDOT Traffic Management Division, Traffic Records Group. This Dataset includes all types of collisions provided by SPA and recorded by Traffic Records from 2004 until present. This data gets weekly updated. In the folling i will be working with the dataset in:


In [6]:
collisions = pd.read_csv("https://s3.us.cloud-object-storage.appdomain.cloud/cf-courses-data/CognitiveClass/DP0701EN/version-2/Data-Collisions.csv")
#pd.set_option('display.max_columns', None)
print("Data read into dataframe!") # takes about 30 seconds

/opt/conda/envs/Python36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3020: DtypeWarning: Columns (33) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Data read into dataframe!


In [7]:
collisions.head()

,SEVERITYCODE,X,Y,OBJECTID,INCKEY,COLDETKEY,REPORTNO,STATUS,ADDRTYPE,INTKEY,...,ROADCOND,LIGHTCOND,PEDROWNOTGRNT,SDOTCOLNUM,SPEEDING,ST_COLCODE,ST_COLDESC,SEGLANEKEY,CROSSWALKKEY,HITPARKEDCAR
0,2,-122.323148,47.703140,1,1307,1307,3502005,Matched,Intersection,37475.0,...,Wet,Daylight,NaN,NaN,NaN,10,Entering at angle,0,0,N
1,1,-122.347294,47.647172,2,52200,52200,2607959,Matched,Block,NaN,...,Wet,Dark - Street Lights On,NaN,6354039.0,NaN,11,From same direction - both going straight - bo...,0,0,N
2,1,-122.334540,47.607871,3,26700,26700,1482393,Matched,Block,NaN,...,Dry,Daylight,NaN,4323031.0,NaN,32,One parked--one moving,0,0,N
3,1,-122.334803,47.604803,4,1144,1144,3503937,Matched,Block,NaN,...,Dry,Daylight,NaN,NaN,NaN,23,From same direction - all others,0,0,N
4,2,-122.306426,47.545739,5,17700,17700,1807429,Matched,Intersection,34387.0,...,Wet,Daylight,NaN,4028032.0,NaN,10,Entering at angle,0,0,N


In [8]:
collisions.shape

(194673, 38)

Our imported dataset consist of the labeled data column 'severity code' and 37 attributes that might be useful for the later prediction. Later on i will decide what to keep and what to drop. In total we have 194673 rows which seems to be a good amount of data.

Each row represents a accident/collisions reported, and for each accident/collision, the corresponding severity code is documented. The other attributes contain information on i.e. weather/road/light conditions, the location, speeding etc.

Let's take a look a specifiy row (objext id 3). From the Data we can see that an accident took place during daylight (see attribute - LIGHTCOND) and the road was dry (see - attribute ROADCOND). The Severitycode (see attribute SEVERITYCODE) for this accident was labeled 1—prop damage , which in this case means : Proporty Damage Only Collision (see attribute SEVERITYDESC). 

In [9]:
collisions.dtypes

SEVERITYCODE        int64
X                 float64
Y                 float64
OBJECTID            int64
INCKEY              int64
COLDETKEY           int64
REPORTNO           object
STATUS             object
ADDRTYPE           object
INTKEY            float64
LOCATION           object
EXCEPTRSNCODE      object
EXCEPTRSNDESC      object
SEVERITYCODE.1      int64
SEVERITYDESC       object
COLLISIONTYPE      object
PERSONCOUNT         int64
PEDCOUNT            int64
PEDCYLCOUNT         int64
VEHCOUNT            int64
INCDATE            object
INCDTTM            object
JUNCTIONTYPE       object
SDOT_COLCODE        int64
SDOT_COLDESC       object
INATTENTIONIND     object
UNDERINFL          object
WEATHER            object
ROADCOND           object
LIGHTCOND          object
PEDROWNOTGRNT      object
SDOTCOLNUM        float64
SPEEDING           object
ST_COLCODE         object
ST_COLDESC         object
SEGLANEKEY          int64
CROSSWALKKEY        int64
HITPARKEDCAR       object
dtype: objec

Concering the datatypes we can see that our dataset has numerical (float64 or int64 types) or categorical (object type) data types. In Addition some attributes have missing data (NaN) as well.

In [ ]:
collisions['SEVERITYCODE'].value_counts()

1    136485
2     58188
Name: SEVERITYCODE, dtype: int64

In [ ]:
collisions['SEVERITYDESC'].value_counts().to_frame()

In [ ]:
ax = collisions['SEVERITYCODE'].plot.bar(y='SEVERITYCODE', rot = 0)

As we can see our labeled data (SEVERITYCODE) is slightly unbalanced. We have the double amount of data with a severity code 1 (136485) im comparison to severity code 2 (58188). This might cause problems for the Modeling and the prediciton later on. I will adress this issue in the data preparation section.

# Data Preparation

The data preparation includes all the required activities to construct the final dataset which will be fed into the modeling tools. Data preparation can be performed multiple times and it includes balancing the labeled data, transformation, filling missing data, and cleaning the dataset.

Select relevant attributes for further Data Preparation

Drop Irrelevant Attributes

I dropped Speeding entirely because it is missing over 180,000 values and this can hamper the results.

Upon further inspection, I found out that ROADCOND and WEATHER have unknown values. This will again hamper the analysis therefore I dropped the values where there is no information

Work with balanced dataset

# Modeling
In this phase, various algorithms and methods can be selected and applied to build the model including supervised machine learning techniques. You can select SVM, XGBoost, decision tree, or any other techniques. You can select a single or multiple machine learning models for the same data mining problem. At this phase, stepping back to the data preparation phase is often required.

# Evaluation
Before proceeding to the deployment stage, the model needs to be evaluated thoroughly to ensure that the business or the applications' objectives are achieved. Certain metrics can be used for the model evaluation such as accuracy, recall, F1-score, precision, and others.
Deployment:

# Deployment

The deployment phase requirements vary from project to project. It can be as simple as creating a report, developing interactive visualization, or making the machine learning model available in the production environment. In this environment, the customers or end-users can utilize the model in different ways such as API, website, or so on.